In [ ]:
import numpy as np

import GPyOpt
import GPy

import sys
sys.path.append("..")

from f16lib.acas import AcasShieldScenario, AcasScenario, AcasScenarioViewer

import csaf.utils as csafutils


import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import f16lib.models.predictor as predictor

In [ ]:
AcasShieldScenario.system_type.components["predictor"].flows["outputs"] = predictor.model_output
AcasShieldScenario.system_type.components["predictor"].initialize = predictor.model_init

In [ ]:
from f16lib.components import f16_xequil
import f16lib.systems as f16sys
import csaf
import typing
import numpy as np


In [ ]:
def get_system(states, sys=AcasShieldScenario):
    """construct a scenario and system given the configuration coordinates (states)"""
    # scenario
    scen = sys(
        [-2000.0, 6000], # balloon position
        750.0, # ownship airspeed
        ((0.0, 21000.0, 1000.0),), # own waypoints
        ((*states[:2], 1000.0),) # intruder waypoints -- none for now
    )
    # scenario, system
    return scen, scen.create_system(
        [*states[:2], # relative position
         states[2], # relative heading
         states[3]]) # relative airspeed


states = [ 7.81833992e+03,  
        1.59743916e+03, 
        -3.66099248e-01, 
        -8.32456483e+01]

scen, my_system = get_system(states)

uscen, u_system = get_system(states, sys=AcasScenario)

In [ ]:
x, y, z = my_system.components["intruder_autopilot"].default_parameters["waypoints"][0]

In [ ]:
my_system.set_component_param("predictor", 
                              "intruder_waypoints",
                             ((x, y, z),))
my_system.set_component_param("predictor", 
                              "own_waypoints",
                             ((0.0, 22000.0, 1000.0),))

In [ ]:
my_system.component_instances["predictor"].parameters["own_waypoints"]

In [ ]:
# check the system
my_system.check()

# view the system
csafutils.view_block_diagram(my_system, ipython_notebook=True)

In [ ]:
trajs = my_system.simulate_tspan((0.0, 20.0), show_status=True)

In [ ]:
#plt.plot(np.array(trajs["acas_out"].outputs)[:, 1])
#plt.plot(np.array(trajs["acas_out"].outputs)[:, 2])
plt.plot(np.array(trajs["acas_out"].times), np.array(trajs["acas_out"].outputs)[:, -1], 'k')
plt.plot(np.array(trajs["acas"].times), np.array(trajs["acas"].outputs)[:, -1], '--b', alpha=0.5)
plt.plot(np.array(trajs["acas_recovery"].times), np.array(trajs["acas_recovery"].outputs)[:, -1], '--r', alpha=0.5)


plt.plot(np.array(trajs["llc"].times), np.array(trajs["llc"].outputs)[:, -1])

In [ ]:
viewer = AcasScenarioViewer(trajs, scen)
fig, ax = viewer.summary_plot()
plt.show()

In [ ]:
from IPython.display import HTML
from matplotlib import animation
anim = viewer.summary_video()
writervideo = animation.FFMpegWriter(fps=30) 
anim.save("bopt_example.mp4", writer=writervideo)
#HTML(anim.to_jshtml())